<font size=6> The total sovereign emissions calculated by adding up scope 1, 2 and 3 emissions  </font>

In [1]:
from pcaf_env import *
import pycountry
import numpy as np

report_currency = 'USD'

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev
create schema if not exists osc_datacommons_dev.mdt_sandbox


In [2]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_table = 'sf_total_sovereign_emissions'

src_table_1 = 'sf_oecd_imgr_fco2'
src_table_2 = 'sf_unfccc_results'
src_table_3 = 'sf_oecd_exgr_dco2'
src_table_4 = 'sf_oecd_exch_rates'
src_table_5 = 'sf_wdi_population'

models_table = 'pcaf_dbt_models'

In [3]:
# Get the values saved in Trino related with Foreign CO2 emissions embodied in gross imports

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1}_source """ 
df1 = pd.read_sql(sql, engine)
df1 = requantify_df(df1)
print("data from SQL:")
display(df1)
# convert to CO2 units to t
df1['value'] = df1['value'].pint.to("t CO2e")
df1 = dequantify_df(df1)
print("data after normalizing to t CO2e")
df1

data from SQL:


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value
0,HRV,WLD,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,7.268
1,HRV,WLD,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,8.178
2,HRV,WLD,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,9.965
3,HRV,WLD,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,8.579
4,HRV,WLD,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,9.028
...,...,...,...,...,...,...
334651,USA,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0
334652,USA,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0
334653,USA,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0
334654,USA,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0


data after normalizing to t CO2e


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,HRV,WLD,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,7268000.0,CO2eq * t
1,HRV,WLD,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,8178000.0,CO2eq * t
2,HRV,WLD,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,9965000.0,CO2eq * t
3,HRV,WLD,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,8579000.0,CO2eq * t
4,HRV,WLD,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,9028000.0,CO2eq * t
...,...,...,...,...,...,...,...
334651,USA,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
334652,USA,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
334653,USA,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
334654,USA,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t


In [4]:
sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3}_source
where industry_code='DTOTAL'""" 
df_exgr = pd.read_sql(sql, engine)
df_exgr = requantify_df(df_exgr)
# convert to CO2 units to t
df_exgr['value'] = df_exgr['value'].pint.to("t CO2e")
df_exgr = dequantify_df(df_exgr)
df_exgr

,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,PER,WLD,DTOTAL,1995,Domestic CO2 emissions embodied in gross exports,2681000.0,CO2eq * t
1,PER,WLD,DTOTAL,1996,Domestic CO2 emissions embodied in gross exports,3154000.0,CO2eq * t
2,PER,WLD,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,3379000.0,CO2eq * t
3,PER,WLD,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,3239000.0,CO2eq * t
4,PER,WLD,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,4167000.0,CO2eq * t
...,...,...,...,...,...,...,...
167323,G20,ZOTH,DTOTAL,2014,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167324,G20,ZOTH,DTOTAL,2015,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167325,G20,ZOTH,DTOTAL,2016,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167326,G20,ZOTH,DTOTAL,2017,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t


In [5]:
###########################

sql=f"""
select validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_4}_source where country_iso_code='DEU'""" 
df_exch_eur = pd.read_sql(sql, engine)
df_exch_eur

,validity_date,value
0,1950,2.144861
1,1951,2.144861
2,1952,2.144861
3,1953,2.147426
4,1954,2.147426
...,...,...
67,2017,0.885206
68,2018,0.846773
69,2019,0.893276
70,2020,0.875506


In [6]:
sql=f"""
select country_iso_code,validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_5}_source """ 
df_population = pd.read_sql(sql, engine)
df_population["value"] = pd.to_numeric(df_population["value"], errors='coerce')
df_population["validity_date"] = pd.to_numeric(df_population["validity_date"], errors='coerce')
df_population=df_population.convert_dtypes()
df_population
#df_population.info(verbose=True)


,country_iso_code,validity_date,value
0,AFG,2017,36296111
1,ALB,2017,2873457
2,DZA,2017,41389174
3,ASM,2017,55617
4,AND,2017,76997
...,...,...,...
3187,SSF,2021,1165563987
3188,SSA,2021,1165464785
3189,TSS,2021,1165563987
3190,UMC,2021,2501427939


<font size=3> Remove invalid country codes </font>

In [7]:
df_country = pd.DataFrame([ country.alpha_3 for country in pycountry.countries ] + ['ROW'], columns=['country_iso_code'], dtype='string')
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code']).convert_dtypes()
print("dat with only validated country names")
display(df1)

df_exgr = pd.merge(df_exgr,df_country,on=['country_iso_code']).convert_dtypes()
df_exgr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    string
dtypes: string(1)
memory usage: 2.1 KB
dat with only validated country names


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,HRV,WLD,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,7268000.0,CO2eq * t
1,HRV,WLD,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,8178000.0,CO2eq * t
2,HRV,WLD,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,9965000.0,CO2eq * t
3,HRV,WLD,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,8579000.0,CO2eq * t
4,HRV,WLD,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,9028000.0,CO2eq * t
...,...,...,...,...,...,...,...
270139,USA,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
270140,USA,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
270141,USA,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t
270142,USA,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0,CO2eq * t


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,PER,WLD,DTOTAL,1995,Domestic CO2 emissions embodied in gross exports,2681000.0,CO2eq * t
1,PER,WLD,DTOTAL,1996,Domestic CO2 emissions embodied in gross exports,3154000.0,CO2eq * t
2,PER,WLD,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,3379000.0,CO2eq * t
3,PER,WLD,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,3239000.0,CO2eq * t
4,PER,WLD,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,4167000.0,CO2eq * t
...,...,...,...,...,...,...,...
135067,USA,MMR,DTOTAL,2014,Domestic CO2 emissions embodied in gross exports,49000.0,CO2eq * t
135068,USA,MMR,DTOTAL,2015,Domestic CO2 emissions embodied in gross exports,55000.0,CO2eq * t
135069,USA,MMR,DTOTAL,2016,Domestic CO2 emissions embodied in gross exports,83000.0,CO2eq * t
135070,USA,MMR,DTOTAL,2017,Domestic CO2 emissions embodied in gross exports,95000.0,CO2eq * t


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [8]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
display(df1)

######################

df_exgr = df_exgr.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df_exgr.convert_dtypes()

df_exgr.info(verbose=True)
df_exgr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB


,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Foreign CO2 emissions embodied in gross imports,D35,APEC,1995,39478000.0,CO2eq * t
1,Foreign CO2 emissions embodied in gross imports,D35,APEC,1996,39637000.0,CO2eq * t
2,Foreign CO2 emissions embodied in gross imports,D35,APEC,1997,46108000.0,CO2eq * t
3,Foreign CO2 emissions embodied in gross imports,D35,APEC,1998,47416000.0,CO2eq * t
4,Foreign CO2 emissions embodied in gross imports,D35,APEC,1999,47128000.0,CO2eq * t
...,...,...,...,...,...,...
4027,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2014,209262000.0,CO2eq * t
4028,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2015,214719000.0,CO2eq * t
4029,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2016,206428000.0,CO2eq * t
4030,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2017,209894000.0,CO2eq * t


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         2016 non-null   string 
 1   industry_code     2016 non-null   string 
 2   country_iso_code  2016 non-null   string 
 3   validity_date     2016 non-null   Int64  
 4   total             2016 non-null   Float64
 5   total_units       2016 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 98.6 KB


,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1995,2008843000.0,CO2eq * t
1,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1996,2055450000.0,CO2eq * t
2,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1997,2151870000.0,CO2eq * t
3,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1998,2177243000.0,CO2eq * t
4,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1999,2300588000.0,CO2eq * t
...,...,...,...,...,...,...
2011,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2014,266081000.0,CO2eq * t
2012,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2015,231751000.0,CO2eq * t
2013,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2016,211686000.0,CO2eq * t
2014,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2017,226732000.0,CO2eq * t


In [9]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date']).convert_dtypes()

#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

df_result = pd.merge(df_result,df_exgr,on=['country_iso_code','validity_date']).convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total':'exported_emissions','total_units':'exported_emissions_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result.drop(columns=['attribute','industry_code'],inplace=True)
df_result.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country_iso_code          2016 non-null   string 
 1   validity_date             2016 non-null   Int64  
 2   scope2_value              2016 non-null   Float64
 3   scope2_value_units        2016 non-null   string 
 4   scope3_value              2016 non-null   Float64
 5   scope3_value_units        2016 non-null   string 
 6   exported_emissions        2016 non-null   Float64
 7   exported_emissions_units  2016 non-null   string 
dtypes: Float64(3), Int64(1), string(4)
memory usage: 134.0 KB


In [10]:
# Get the values saved in Trino containing  GHG emissions provided by UNFCCC & GDP PPP values from Worldbank

sql=f"""
select country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,ghg_total_with_lulucf,ghg_total_with_lulucf_units
,gdp,gdp_units,gdp_ppp,gdp_ppp_units
from {ingest_catalog}.{ingest_schema}.{src_table_2}_source """ 
df_unfccc = pd.read_sql(sql, engine)

if (report_currency == 'EUR'):
    df_unfccc = pd.merge(df_unfccc,df_exch_eur,on=['validity_date'],how='inner')   
    df_unfccc['gdp'] = df_unfccc['gdp'] * df_unfccc['value']
    df_unfccc['gdp_ppp'] = df_unfccc['gdp_ppp'] * df_unfccc['value']
    df_unfccc['gdp_units'] = 'EUR'
    df_unfccc['gdp_ppp_units'] = 'EUR'
    df_unfccc.drop(columns=['value'],inplace=True)


df_unfccc[df_unfccc['country_iso_code']=='ARG']
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                15025 non-null  string 
 1   validity_date                   15025 non-null  Int64  
 2   ghg_total_without_lulucf        10633 non-null  Float64
 3   ghg_total_without_lulucf_units  15025 non-null  string 
 4   scope1_excl_source              10633 non-null  string 
 5   ghg_total_with_lulucf           2256 non-null   Float64
 6   ghg_total_with_lulucf_units     15025 non-null  string 
 7   gdp                             13118 non-null  Float64
 8   gdp_units                       15025 non-null  string 
 9   gdp_ppp                         7472 non-null   Float64
 10  gdp_ppp_units                   15025 non-null  string 
dtypes: Float64(4), Int64(1), string(6)
memory usage: 1.3 MB


In [11]:
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)

df_unfccc.fillna(value={'ghg_total_without_lulucf':0, 'ghg_total_with_lulucf':0, 'gdp':0, 'gdp_ppp':0}, inplace=True)

df_unfccc = requantify_df(df_unfccc)
df_unfccc.info(verbose=True)
# convert to CO2 units to Mt
df_unfccc['ghg_total_without_lulucf'] = df_unfccc['ghg_total_without_lulucf'].pint.to("t CO2e")
df_unfccc['ghg_total_with_lulucf'] = df_unfccc['ghg_total_with_lulucf'].pint.to("t CO2e")
####

df_unfccc.info(verbose=True)


df_unfccc[df_unfccc['country_iso_code']=='ARG']



#df_unfccc['gdp'] = df_unfccc['gdp'].pint.to("Millions USD")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                15025 non-null  string 
 1   validity_date                   15025 non-null  Int64  
 2   ghg_total_without_lulucf        10633 non-null  Float64
 3   ghg_total_without_lulucf_units  15025 non-null  string 
 4   scope1_excl_source              10633 non-null  string 
 5   ghg_total_with_lulucf           2256 non-null   Float64
 6   ghg_total_with_lulucf_units     15025 non-null  string 
 7   gdp                             13118 non-null  Float64
 8   gdp_units                       15025 non-null  string 
 9   gdp_ppp                         7472 non-null   Float64
 10  gdp_ppp_units                   15025 non-null  string 
dtypes: Float64(4), Int64(1), string(6)
memory usage: 1.3 MB
<class 'pandas.core.frame.DataFrame'>

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(


,country_iso_code,validity_date,ghg_total_without_lulucf,scope1_excl_source,ghg_total_with_lulucf,gdp,gdp_ppp
1737,ARG,1990,231000000.0,PRIMAP,216291389.487407,141352368714.691,234455344931.543
1771,ARG,2000,280000000.0,PRIMAP,238702903.51909998,284203750000.0,427907425395.381
1772,ARG,2010,321000000.0,PRIMAP,447642430.0,423627422092.49,736799121185.705
1773,ARG,2012,334000000.0,PRIMAP,429478449.93,545982375701.128,819697901873.041
1774,ARG,1994,249000000.0,PRIMAP,223335527.904266,257440000000.0,349669909411.917
...,...,...,...,...,...,...,...
1827,ARG,2017,350000000.0,PRIMAP,0.0,643628665302.155,1039330591568.54
1828,ARG,2018,347000000.0,PRIMAP,0.0,524819742918.669,1036307018234.46
1829,ARG,2019,337000000.0,PRIMAP,0.0,452818426182.658,1033473164885.89
1830,ARG,2020,323000000.0,PRIMAP,0.0,389591035520.675,942431459963.083


In [12]:
#df_unfccc = dequantify_df(df_unfccc)
#df_unfccc
#df_unfccc.info(verbose=True)
#df_unfccc

df_result= requantify_df(df_result)
#df_result= dequantify_df(df_result)
#df_result


/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(


<font size=3><b>Merge UNFCCC , Worldbank and OECD data</b> </font>

In [13]:
df_result = pd.merge(df_result,df_unfccc,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'ghg_total_without_lulucf':'scope1_excl_lulucf',
                  'ghg_total_with_lulucf':'scope1_incl_lulucf'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)

# enrich with population data

df_result = pd.merge(df_result,df_population,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'value':'population'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)
df_result["population"] = df_result["population"].fillna(0).astype('int64')

sum_yn = 1.0 * ((df_result['scope1_excl_lulucf'] > 0) & (df_result['scope2_value'] > 0) &  (df_result['scope3_value'] > 0) )
calc_attr_factor_yn = 1.0 * (df_result['gdp_ppp'] > 0)
population_available = 1.0 * (df_result['population'] > 0)

consumption_partial = df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions']
df_result['consumption_emissions_excl_lulucf'] = sum_yn * ( df_result['scope1_excl_lulucf'] + consumption_partial)
df_result['consumption_emissions_incl_lulucf'] = sum_yn * ( df_result['scope1_incl_lulucf'] + consumption_partial)

population_partial = population_available / df_result['population']
df_result['consumption_emissions_excl_lulucf_per_capita'] = population_partial * df_result['consumption_emissions_excl_lulucf']
df_result['consumption_emissions_incl_lulucf_per_capita'] = population_partial * df_result['consumption_emissions_incl_lulucf']


###############
df_result['attribution_factor_scope1_excl_lulucf'] = calc_attr_factor_yn * (df_result['scope1_excl_lulucf']) / df_result['gdp_ppp']
df_result['attribution_factor_scope1_incl_lulucf'] = calc_attr_factor_yn * (df_result['scope1_incl_lulucf']) / df_result['gdp_ppp']

attr_partial = calc_attr_factor_yn * sum_yn
s2s3_partial = df_result['scope2_value'] + df_result['scope3_value']
attr_partial_gdp_ppp = attr_partial / df_result['gdp_ppp']
df_result['attribution_factor_excl_lulucf'] = attr_partial_gdp_ppp * (df_result['scope1_excl_lulucf'] + s2s3_partial)
df_result['attribution_factor_incl_lulucf'] = attr_partial_gdp_ppp * (df_result['scope1_incl_lulucf'] + s2s3_partial)
##########
attr_partial_gdp = attr_partial / df_result['gdp']
df_result['attribution_factor_excl_lulucf_gdp'] = attr_partial_gdp * (df_result['scope1_excl_lulucf'] + s2s3_partial)
df_result['attribution_factor_incl_lulucf_gdp'] = attr_partial_gdp * (df_result['scope1_incl_lulucf'] + s2s3_partial)


#df_result['ghg_total_without_lulucf']=df_result['ghg_total_without_lulucf'].replace(0, np.nan)
#df_result['ghg_total_with_lulucf']=df_result['ghg_total_with_lulucf'].replace(0, np.nan)
#df_result['gdp']=df_result['gdp'].replace(0, np.nan)
#df_result['gdp_ppp']=df_result['gdp_ppp'].replace(0, np.nan)



#df_result

columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_source','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result = df_result.reindex(columns=columns_order)
df_result.info(verbose=True)
df_result= dequantify_df(df_result)
df_result.info(verbose=True)
# remove invalid country_iso_codes (regions, ...)

df_result = pd.merge(df_result,df_country,on=['country_iso_code'])  

cols= ['scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result[cols] = df_result[cols].replace({0:np.nan})


df_result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15433 entries, 0 to 15432
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   country_iso_code    15433 non-null  string         
 1   validity_date       15433 non-null  Int64          
 2   scope2_value        2016 non-null   pint[CO2eq * t]
 3   scope3_value        2016 non-null   pint[CO2eq * t]
 4   exported_emissions  2016 non-null   pint[CO2eq * t]
 5   scope1_excl_lulucf  15025 non-null  pint[CO2eq * t]
 6   scope1_excl_source  10633 non-null  string         
 7   scope1_incl_lulucf  15025 non-null  pint[CO2eq * t]
 8   gdp                 15025 non-null  pint[USD]      
 9   gdp_ppp             15025 non-null  pint[USD]      
dtypes: Int64(1), pint[CO2eq * t](5), pint[USD](2), string(2)
memory usage: 1.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15536 entries, 0 to 15535
Data columns (total 11 columns):
 #   Column    

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_excl_source,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
0,ARG,1995,251000000.0,CO2eq * t,PRIMAP,NaN,CO2eq * t,21000.0,CO2eq * t,9356000.0,...,NaN,CO2eq * t / USD,0.000751,CO2eq * t / USD,0.001009,CO2eq * t / USD,0.000027,CO2eq * t / USD,0.000036,CO2eq * t / USD
1,ARG,1996,264000000.0,CO2eq * t,PRIMAP,NaN,CO2eq * t,28000.0,CO2eq * t,11287000.0,...,NaN,CO2eq * t / USD,0.000739,CO2eq * t / USD,0.001012,CO2eq * t / USD,0.000030,CO2eq * t / USD,0.000042,CO2eq * t / USD
2,ARG,1997,265000000.0,CO2eq * t,PRIMAP,2.419562e+08,CO2eq * t,26000.0,CO2eq * t,14968000.0,...,0.00059,CO2eq * t / USD,0.000683,CO2eq * t / USD,0.000956,CO2eq * t / USD,0.000627,CO2eq * t / USD,0.000877,CO2eq * t / USD
3,ARG,1998,271000000.0,CO2eq * t,PRIMAP,NaN,CO2eq * t,43000.0,CO2eq * t,13397000.0,...,NaN,CO2eq * t / USD,0.000661,CO2eq * t / USD,0.000951,CO2eq * t / USD,0.000031,CO2eq * t / USD,0.000045,CO2eq * t / USD
4,ARG,1999,277000000.0,CO2eq * t,PRIMAP,NaN,CO2eq * t,61000.0,CO2eq * t,12270000.0,...,NaN,CO2eq * t / USD,0.000686,CO2eq * t / USD,0.001020,CO2eq * t / USD,0.000029,CO2eq * t / USD,0.000043,CO2eq * t / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12316,GIB,2020,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
12317,GIB,2019,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
12318,GIB,2012,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
12319,GIB,2014,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD


In [14]:
#df_result['scope1_value_units'] = 'Mt CO2e'
#df_result['scope1_value'] = df_result['scope1_value'].round(decimals = 3)
#pd.options.display.float_format = '{:.3f}'.format
#df_result= df_result.convert_dtypes()
#df_result.info(verbose=True)
df_result[df_result['country_iso_code']=='CAN']
#df_result

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_excl_source,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
414,CAN,1995,6.454553e+08,CO2eq * t,UNFCCC,6.058811e+08,CO2eq * t,5129000.0,CO2eq * t,179111000.0,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
415,CAN,1996,6.667019e+08,CO2eq * t,UNFCCC,6.212992e+08,CO2eq * t,5269000.0,CO2eq * t,188198000.0,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
416,CAN,1997,6.822990e+08,CO2eq * t,UNFCCC,6.329551e+08,CO2eq * t,6362000.0,CO2eq * t,202972000.0,...,0.000838,CO2eq * t / USD,0.001180,CO2eq * t / USD,0.001361,CO2eq * t / USD,0.001115,CO2eq * t / USD,0.001286,CO2eq * t / USD
417,CAN,1998,6.886139e+08,CO2eq * t,UNFCCC,6.335611e+08,CO2eq * t,7993000.0,CO2eq * t,214108000.0,...,0.000798,CO2eq * t / USD,0.001147,CO2eq * t / USD,0.001436,CO2eq * t / USD,0.001078,CO2eq * t / USD,0.001350,CO2eq * t / USD
418,CAN,1999,7.017587e+08,CO2eq * t,UNFCCC,6.540323e+08,CO2eq * t,9100000.0,CO2eq * t,222868000.0,...,0.000773,CO2eq * t / USD,0.001103,CO2eq * t / USD,0.001376,CO2eq * t / USD,0.001047,CO2eq * t / USD,0.001306,CO2eq * t / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,CAN,1986,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
472,CAN,1966,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
473,CAN,1963,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
474,CAN,1978,NaN,CO2eq * t,<NA>,NaN,CO2eq * t,NaN,CO2eq * t,NaN,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD


Calculate Attribution factor = (Scope1+Scope2+Scope3)/GDP


In [15]:


#df_result= requantify_df(df_result)
format_mapper =  {'gdp_ppp': '{0:,.0f}',
           'gdp': '{0:,.0f}',       
           'attribution_factor_excl_lulucf': '{0:.8f}',
           'attribution_factor_incl_lulucf': '{0:.8f}',
           'attribution_factor_excl_lulucf_gdp': '{0:.8f}',
           'attribution_factor_incl_lulucf_gdp': '{0:.8f}',       
           'scope1_excl_lulucf': '{0:,.0f}',       
           'scope1_incl_lulucf': '{0:,.0f}',              
           'scope2_value': '{0:,.0f}',              
           'scope3_value': '{0:,.0f}'
                 }

df_result[df_result['country_iso_code']=='CAN'].style.format(format_mapper)


,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_excl_source,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,exported_emissions,exported_emissions_units,gdp_ppp,gdp_ppp_units,gdp,gdp_units,consumption_emissions_excl_lulucf,consumption_emissions_excl_lulucf_units,consumption_emissions_incl_lulucf,consumption_emissions_incl_lulucf_units,population,consumption_emissions_excl_lulucf_per_capita,consumption_emissions_excl_lulucf_per_capita_units,consumption_emissions_incl_lulucf_per_capita,consumption_emissions_incl_lulucf_per_capita_units,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_excl_lulucf_units,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
414,CAN,1995,"645,455,297",CO2eq * t,UNFCCC,"605,881,079",CO2eq * t,"5,129,000",CO2eq * t,"179,111,000",CO2eq * t,136648000.000000,CO2eq * t,nan,USD,"604,031,623,433",USD,693047296.776335,CO2eq * t,653473079.122202,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
415,CAN,1996,"666,701,910",CO2eq * t,UNFCCC,"621,299,195",CO2eq * t,"5,269,000",CO2eq * t,"188,198,000",CO2eq * t,139744000.000000,CO2eq * t,nan,USD,"628,546,387,972",USD,720424910.230366,CO2eq * t,675022195.292942,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
416,CAN,1997,"682,298,998",CO2eq * t,UNFCCC,"632,955,107",CO2eq * t,"6,362,000",CO2eq * t,"202,972,000",CO2eq * t,160734000.000000,CO2eq * t,"755,390,170,461",USD,"654,986,999,856",USD,730898998.432811,CO2eq * t,681555106.804654,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,0.000903,CO2eq * t / USD,0.000838,CO2eq * t / USD,0.00118036,CO2eq * t / USD,0.00136130,CO2eq * t / USD,0.00111504,CO2eq * t / USD,0.00128596,CO2eq * t / USD
417,CAN,1998,"688,613,931",CO2eq * t,UNFCCC,"633,561,120",CO2eq * t,"7,993,000",CO2eq * t,"214,108,000",CO2eq * t,166623000.000000,CO2eq * t,"793,674,327,704",USD,"634,000,000,000",USD,744091931.024858,CO2eq * t,689039119.917268,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,0.000868,CO2eq * t / USD,0.000798,CO2eq * t / USD,0.00114747,CO2eq * t / USD,0.00143646,CO2eq * t / USD,0.00107810,CO2eq * t / USD,0.00134962,CO2eq * t / USD
418,CAN,1999,"701,758,712",CO2eq * t,UNFCCC,"654,032,317",CO2eq * t,"9,100,000",CO2eq * t,"222,868,000",CO2eq * t,171505000.000000,CO2eq * t,"846,412,945,810",USD,"678,412,196,271",USD,762221711.728973,CO2eq * t,714495316.832600,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,0.000829,CO2eq * t / USD,0.000773,CO2eq * t / USD,0.00110316,CO2eq * t / USD,0.00137634,CO2eq * t / USD,0.00104677,CO2eq * t / USD,0.00130599,CO2eq * t / USD
419,CAN,2000,"726,987,283",CO2eq * t,UNFCCC,"690,491,255",CO2eq * t,"18,191,000",CO2eq * t,"236,036,000",CO2eq * t,178858000.000000,CO2eq * t,"900,996,986,801",USD,"744,773,415,932",USD,802356282.993296,CO2eq * t,765860254.730039,CO2eq * t,30685730.000000,26.147538,CO2eq * t,24.958189,CO2eq * t,0.000807,CO2eq * t / USD,0.000766,CO2eq * t / USD,0.00108903,CO2eq * t / USD,0.00131747,CO2eq * t / USD,0.00104853,CO2eq * t / USD,0.00126846,CO2eq * t / USD
420,CAN,2001,"718,320,043",CO2eq * t,UNFCCC,"673,786,320",CO2eq * t,"19,531,000",CO2eq * t,"228,887,000",CO2eq * t,175811000.000000,CO2eq * t,"937,786,776,177",USD,"738,981,792,355",USD,790927043.152663,CO2eq * t,746393319.668803,CO2eq * t,nan,nan,CO2eq * t,nan,CO2eq * t,0.000766,CO2eq * t / USD,0.000718,CO2eq * t / USD,0.00103087,CO2eq * t / USD,0.00130820,CO2eq * t / USD,0.00098338,CO2eq * t / USD,0.00124794,CO2eq

<font size=3>Save the results in Trino</font>

In [16]:
# Add the sources used for the calculations in the result table
#df_result['scope1_excl_source'] = "UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope1_incl_source'] = "UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope2_source'] = "OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE"
df_result['scope3_source'] = "OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value "          
df_result['gdp_ppp_source'] = "Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv"          


In [17]:
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

df_result['year'] = df_result['validity_date']
#df_result['validity_date'] = pd.to_datetime(df_result.validity_date, format='%Y')
df_result = df_result.convert_dtypes()
#df_result.validity_date = df_result.validity_date.astype('datetime64[ns]')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12321 entries, 0 to 12320
Data columns (total 42 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_iso_code                                    12321 non-null  string 
 1   validity_date                                       12321 non-null  Int64  
 2   scope1_excl_lulucf                                  10633 non-null  Float64
 3   scope1_excl_lulucf_units                            12321 non-null  string 
 4   scope1_excl_source                                  10633 non-null  string 
 5   scope1_incl_lulucf                                  2256 non-null   Float64
 6   scope1_incl_lulucf_units                            12321 non-null  string 
 7   scope2_value                                        1419 non-null   Float64
 8   scope2_value_units                                  12321 non-null  string 


In [18]:
df_result.info("verbose=True")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12321 entries, 0 to 12320
Data columns (total 43 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_iso_code                                    12321 non-null  string 
 1   validity_date                                       12321 non-null  Int64  
 2   scope1_excl_lulucf                                  10633 non-null  Float64
 3   scope1_excl_lulucf_units                            12321 non-null  string 
 4   scope1_excl_source                                  10633 non-null  string 
 5   scope1_incl_lulucf                                  2256 non-null   Float64
 6   scope1_incl_lulucf_units                            12321 non-null  string 
 7   scope2_value                                        1419 non-null   Float64
 8   scope2_value_units                                  12321 non-null  string 


In [19]:
df_result = df_result[df_result['year']>=1990]
df_result.sort_values(by=['country_iso_code','validity_date'], inplace=True)
df_result.to_excel("pcaf_results.xlsx",index=False)
df_result.to_csv("pcaf_results.csv",index=False,encoding='utf-8')


In [ ]:
try:
    os.mkdir("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models", mode=0o755)
except FileExistsError:
    pass
shutil.rmtree("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models/.ipynb_checkpoints", ignore_errors=True)

In [ ]:
custom_meta_content = {
    'data provider': df[['data_provider']],
    'title': 'UNFCCC sovereign emissions by country and year',
    'author': 'UNFCCC',
    'contact': 'GHGdata@unfccc.int',
    'description': json.dumps("""
UNFCCC sovereign emissions by country and year"""),
    'release_date': '2022-01-01 00:00:00',
    # How should we describe our transformative step here?
}

def description_is (s):
    return { 'Description': json.dumps(s)}

custom_meta_fields = {}
custom_meta_fields['country_iso_code'] = description_is("ISO-3166 Country Code (alpha_3)")
custom_meta_fields['validity_date'] = description_is("Year of measurement")
# custom_meta_fields['country_name'] = description_is("ISO-3166 Country Name")
custom_meta_fields['year'] = description_is("Year of measurement")
custom_meta_fields['scope1_excl_lulucf'] = description_is("if not NULL, Scope 1 emissions excluding LULUCF")
custom_meta_fields['scope1_excl_lulucf_units'] = description_is("kt CO2eq")
custom_meta_fields['scope1_excl_source'] = description_is("UNFCC or PRIMAP or NULL")
custom_meta_fields['scope1_incl_lulucf'] = description_is("if not NULL, Scope 1 emissions including LULUCF")
custom_meta_fields['scope1_incl_lulucf_units'] = description_is("kt CO2eq")
custom_meta_fields['scope1_incl_source'] = description_is("UNFCC or PRIMAP or NULL")
custom_meta_fields['scope2_value'] = description_is("if not NULL, Scope 2 emissions")
custom_meta_fields['scope2_value_units'] = description_is("kt CO2eq")
custom_meta_fields['scope2_source'] = description_is("OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE")
custom_meta_fields['scope3_value'] = description_is("if not NULL, Scope 3 emissions")
custom_meta_fields['scope3_value_units'] = description_is("kt CO2eq")
custom_meta_fields['scope3_source'] = description_is("OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value")
custom_meta_fields['exported_emissions'] = description_is("if not NULL, exported emissions")
custom_meta_fields['exported_emissions_units'] = description_is("kt CO2eq")
custom_meta_fields['gdp'] = description_is("Gross Domestic Product")
custom_meta_fields['gdp_units'] = description_is("USD")
custom_meta_fields['gdp_ppp'] = description_is("Gross Domestic Product, normalized to Purchasing Power Parity")
custom_meta_fields['gdp_ppp_units'] = description_is("USD")
custom_meta_fields['gdp_ppp_source'] = description_is("Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv")
custom_meta_fields['consumption_emissions_excl_lulucf'] = description_is("consumption emissions excluding LULUCH")
custom_meta_fields['consumption_emissions_excl_lulucf_units'] = description_is("kt CO2e / USD")
custom_meta_fields['consumption_emissions_incl_lulucf'] = description_is("consumption emissions including LULUCH")
custom_meta_fields['consumption_emissions_incl_lulucf_unit'] = description_is("kt CO2e / USD")
custom_meta_fields['population'] = description_is("Population")
custom_meta_fields['attribution_factor_scope1_excl_lulucf'] = description_is("if not NULL, Scope 1 emissions excluding LULUCF per USD")
custom_meta_fields['attribution_factor_scope1_excl_lulucf_units'] = description_is("t CO2eq / USD")
custom_meta_fields['attribution_factor_scope1_incl_lulucf'] = description_is("if not NULL, Scope 1 emissions including LULUCF per USD")
custom_meta_fields['attribution_factor_scope1_incl_lulucf_units'] = description_is("t CO2eq / USD")
custom_meta_fields['attribution_factor_excl_lulucf'] = description_is("if not NULL, emissions excluding LULUCF per USD")
custom_meta_fields['attribution_factor_excl_lulucf_units'] = description_is("t CO2eq / USD")
custom_meta_fields['attribution_factor_excl_lulucf_gdp'] = description_is("if not NULL, emissions excluding LULUCF per USD")
custom_meta_fields['attribution_factor_excl_lulucf_gdp_units'] = description_is("t CO2eq / USD")
custom_meta_fields['attribution_factor_incl_lulucf'] = description_is("if not NULL, emissions including LULUCF per USD")
custom_meta_fields['attribution_factor_incl_lulucf_units'] = description_is("t CO2eq / USD")
custom_meta_fields['attribution_factor_incl_lulucf_gdp'] = description_is("if not NULL, emissions including LULUCF per USD")
custom_meta_fields['attribution_factor_incl_lulucf_gdp_units'] = description_is("t CO2eq / USD")

for f in custom_meta_fields:
    if 'ISO' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['ISO']
    elif 'IPCC' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['IPCC']
    elif 'WDI' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['WDI']
    elif f == 'year':
        custom_meta_fields[f]['tags'] = ['annual']

rec_source
data_provider
country_iso_code
country_name
attribute
value
value_units
validity_date


In [20]:
create_trino_table_and_dbt_metadata(ingest_table, df_result, ['country_iso_code'], custom_meta_content, custom_meta_fields, verbose=True)

drop table if exists mdt_sandbox.sf_total_sovereign_emissions_source
enforcing dataframe partition column order

verifying existence of table osc_datacommons_dev.mdt_sandbox.sf_total_sovereign_emissions_source
create table if not exists osc_datacommons_dev.mdt_sandbox.sf_total_sovereign_emissions_source (
    validity_date bigint,
    scope1_excl_lulucf double,
    scope1_excl_lulucf_units varchar,
    scope1_excl_source varchar,
    scope1_incl_lulucf double,
    scope1_incl_lulucf_units varchar,
    scope2_value double,
    scope2_value_units varchar,
    scope3_value bigint,
    scope3_value_units varchar,
    exported_emissions bigint,
    exported_emissions_units varchar,
    gdp_ppp double,
    gdp_ppp_units varchar,
    gdp double,
    gdp_units varchar,
    consumption_emissions_excl_lulucf double,
    consumption_emissions_excl_lulucf_units varchar,
    consumption_emissions_incl_lulucf double,
    consumption_emissions_incl_lulucf_units varchar,
    population bigint,
    con

In [21]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}_source where country_iso_code = 'BRA' and year > 2017 """
pd.read_sql(sql, engine)

,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_excl_source,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units,scope1_incl_source,scope2_source,scope3_source,gdp_ppp_source,year,country_iso_code
0,2021,1.130000e+09,CO2eq * t,PRIMAP,None,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2021,BRA
1,2018,1.090000e+09,CO2eq * t,PRIMAP,None,CO2eq * t,11000.0,CO2eq * t,114203000.0,CO2eq * t,...,0.000036,CO2eq * t / USD,0.00006,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2018,BRA
2,2019,1.100000e+09,CO2eq * t,PRIMAP,None,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2019,BRA
3,2020,1.080000e+09,CO2eq * t,PRIMAP,None,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2020,BRA


In [2]:
osc._do_sql(f"insert into {ingest_schema}.{models_table} values(\'{json.dumps(dbt_dict['models'])}\')", engine, verbose=True)

NameError: name 'osc' is not defined

In [17]:
qres = osc._do_sql(f"select * from {ingest_schema}.{models_table}", engine)

In [26]:
dbt_dict['models'] = { k:v for row in qres for k,v in json.loads(row[0]).items() }

In [27]:
dbt_yml = open("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models/pcaf_schema.yml", "w", encoding="utf-8")

In [28]:
print("version: 2", file=dbt_yml)

indent = 0
print("\nmodels:", file=dbt_yml)
indent = indent + 2
for name in dbt_dict['models']:
    model = dbt_dict['models'][name]
    print(f"{' '*indent}- name: {name}", file=dbt_yml)
    indent = indent + 2
    print(f"{' '*indent}description: {model['description']}", file=dbt_yml)
    print(f"\n{' '*indent}columns:", file=dbt_yml)
    indent = indent + 2
    columns = model['columns']
    for col in columns:
        print(f"{' '*indent}- name: {col}", file=dbt_yml)
        indent = indent + 2
        for col_meta in columns[col].keys():
            print(f"{' '*indent}{col_meta}: {columns[col][col_meta]}", file=dbt_yml)
        indent = indent - 2
    print("", file=dbt_yml) # newline comes for free...
    indent = indent - 4
indent = indent - 2
assert(indent==0)

In [29]:
dbt_yml.close()